# Preprocessing

In [1]:
import pandas as pd
import numpy as np
import spacy
import nltk
import re
from nltk import word_tokenize, sent_tokenize, text
from collections import Counter
pd.set_option("display.max_columns", None)

In [2]:
def preprocess(text):
    '''
    This function takes a string of text and does the following:
    - Remove initial "*" characters -- these are useless and extra noise
    - Convert alphabetic characters to lower case (Hello --> hello)
    - Replace numeric characters with "*" character.
    '''
    text = text.replace('*','').lower()
    text = re.sub('\d', '*', text)
    return text

def create_vocabulary(counter, threshold):
    '''
    @counter is a Counter object listing the number of times each token appears in the corpus
    @threshold = number of times a token must appear in corpus to be listed in vocabulary 
    '''
    Vocabulary = []
    for key in counter:
        value = counter[key]
        if value>=threshold:
            Vocabulary.append(key)
    vocab_indexer = dict()
    index_value = 1
    for word in Vocabulary:
        vocab_indexer[word] = index_value
        index_value = index_value+1
    return vocab_indexer

def token_to_index(tokens, token_indexer):
    """
    Function that transform a list of tokens in a document and coverts it to a list of token indices.
    @param tokens: list of tokens from document
    @param token_indexer: dictionary that maps each token in the vocabulary to an unique index
    """
    # Please DO NOT assign any ngram to index 0 which is reserved for PAD token
    index_list = []
    for token in tokens:
        try:  
            index_list.append(token_indexer[token])
        except:
            pass
    return index_list

--------

# MIMIC III 

## Create a dictionary that links each HADM_ID to a list of ICD Codes 

Subject_ID can have multiple HADM_IDs (unique hospital admissions), but not the other way around. 

In [3]:
Diagnoses_ICD = pd.read_csv('./MIMIC-III/DIAGNOSES_ICD.csv')
Diagnoses_ICD = Diagnoses_ICD[-Diagnoses_ICD['ICD9_CODE'].isnull()]
Diagnoses_ICD['Rolled_ICD'] = Diagnoses_ICD['ICD9_CODE'].str[0:3]

NumberCodes = len(Diagnoses_ICD['ICD9_CODE'].unique())
NumberRolled = len(Diagnoses_ICD['Rolled_ICD'].unique())

print('Unique ICD-9 codes:', NumberCodes, '\nUnique 3-Digit Codes:',NumberRolled)
print('HADM_IDs', len(Diagnoses_ICD['HADM_ID'].unique()), '\nSUBJECT_IDs', 
                     len(Diagnoses_ICD['SUBJECT_ID'].unique()))

Diagnoses_ICD.head()

Unique ICD-9 codes: 6984 
Unique 3-Digit Codes: 942
HADM_IDs 58929 
SUBJECT_IDs 46517


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,Rolled_ICD
0,1297,109,172335,1.0,40301,403
1,1298,109,172335,2.0,486,486
2,1299,109,172335,3.0,58281,582
3,1300,109,172335,4.0,5855,585
4,1301,109,172335,5.0,4254,425


In [4]:
HADMID_Code_Dict = dict()
Unique_Visits = Diagnoses_ICD['HADM_ID'].unique()
for visit in Unique_Visits:
    VisitDF = Diagnoses_ICD[Diagnoses_ICD['HADM_ID']==visit].reset_index()
    ListOfICDs=[]
    for i in range(len(VisitDF)):
        ListOfICDs.append(VisitDF.loc[i, 'ICD9_CODE'])
    HADMID_Code_Dict[visit] = ListOfICDs

In [5]:
HADMID_Code_Dict[172335]

['40301',
 '486',
 '58281',
 '5855',
 '4254',
 '2762',
 '7100',
 '2767',
 '7243',
 '45829',
 '2875',
 '28521',
 '28529',
 '27541']

--------

## Create a dictionary that links each HADM_ID to a Discharge Summary 

In [5]:
NoteEvents = pd.read_csv('./MIMIC-III/NOTEEVENTS.csv')
Notes = NoteEvents[NoteEvents['CATEGORY'] == 'Discharge summary'].reset_index(drop=True)
Notes = Notes[['SUBJECT_ID','HADM_ID','CHARTDATE','DESCRIPTION', 'TEXT']]
Notes.head()

/Users/Brenton/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SUBJECT_ID,HADM_ID,CHARTDATE,DESCRIPTION,TEXT
0,22532,167853.0,2151-08-04,Report,Admission Date: [**2151-7-16**] Dischar...
1,13702,107527.0,2118-06-14,Report,Admission Date: [**2118-6-2**] Discharg...
2,13702,167118.0,2119-05-25,Report,Admission Date: [**2119-5-4**] D...
3,13702,196489.0,2124-08-18,Report,Admission Date: [**2124-7-21**] ...
4,26880,135453.0,2162-03-25,Report,Admission Date: [**2162-3-3**] D...


### One problem we face here is that a HADM_ID might be associated with multiple discharge summaries. 

More specifically, an HADM_ID might have one "Report" and several "Addendums". Try to make it so that each HADM_ID is associated with EXACTLY one block of text. In order to do so, we'll need to concatenate texts for each HADM_ID.

In [6]:
NotesPerID = Notes.groupby('HADM_ID').count()[['TEXT']]
HADM_IDs_with_MultipleCounts = NotesPerID[NotesPerID['TEXT'] > 1].index
HADM_IDs_with_MultipleCounts #These are the HADM_IDs with more than one block of text.

Float64Index([100006.0, 100031.0, 100036.0, 100041.0, 100112.0, 100132.0,
              100136.0, 100137.0, 100138.0, 100153.0,
              ...
              199883.0, 199907.0, 199917.0, 199918.0, 199951.0, 199952.0,
              199961.0, 199962.0, 199964.0, 199988.0],
             dtype='float64', name='HADM_ID', length=5720)

In [13]:
#NotesToAppend: rows of text that need to be merged according to HADM_ID
NotesToAppend = Notes[Notes['HADM_ID'].isin(HADM_IDs_with_MultipleCounts)]

UnchangedNotes = Notes[-Notes['HADM_ID'].isin(HADM_IDs_with_MultipleCounts)]
print(len(NotesToAppend), len(UnchangedNotes))

12646 47006


In [11]:
ChangedNotes = pd.DataFrame()
for ID in HADM_IDs_with_MultipleCounts: #For each HADM_ID with multiple texts
    subdf = NotesToAppend[NotesToAppend['HADM_ID'] == ID] #create a smaller df with only that HADMID
    combined_text = subdf['TEXT'].str.cat() #combine all text in that column into one entry 
    subdf = subdf.drop_duplicates(subset='HADM_ID') #turn smaller df into 1-row df
    subdf['TEXT'] = combined_text 
    ChangedNotes = ChangedNotes.append(subdf)

In [12]:
TotalNotes = ChangedNotes.append(UnchangedNotes).reset_index(drop=True)
print(len(TotalNotes), len(TotalNotes['HADM_ID'].unique()))

52726 52726


### Preprocess text. Then count the number of times each token appears in the corpus. 

In [17]:
'''
Preprocessing Steps
- Convert all alphabetical characters to lower case
- Eliminate any '*' characters that do not correspond to numbers
- Convert numeric characters to '*' -- so "2/2/2011" becomes "*/*/****"
'''

'\nPreprocessing Steps\n- Convert all alphabetical characters to lower case\n- Eliminate any \'*\' characters that do not correspond to numbers\n- Convert numeric characters to \'*\' -- so "2/2/2011" becomes "*/*/****"\n'

In [14]:
Corpus_Counts = Counter()
for i in range(len(TotalNotes)): #for each discharge summary
    text = TotalNotes['TEXT'][i]
    text_new = preprocess(text) #process text
    tokens = word_tokenize(text_new) #tokenize
    unigram_counts = Counter(tokens) #count number of unigrams
    Corpus_Counts.update(unigram_counts) #update the Counter for each discharge summary

In [5]:
#Corpus_Counts

Counter({'pneumoconiosis': 5,
         'pain/fatigue/doe': 1,
         '**/**.his': 1,
         'moncytic': 4,
         'low-phosphate': 1,
         'w/ugib': 8,
         'anti-retroviarals': 1,
         'spray/nostril': 5,
         'noc': 15,
         'coarseness': 12,
         'pa/lateral': 111,
         'anterior/anterior-septal': 1,
         'tra*p': 1,
         'hmf-': 1,
         'mg/kilo/min': 1,
         'iliac-left': 1,
         'pao*/fio*': 29,
         '-shortgut': 1,
         'aavandia': 1,
         'saliene': 1,
         'acquired-pneumonia': 2,
         'over-breathing': 11,
         'septdown': 1,
         'lbut': 1,
         'spep/upep': 94,
         'junction/superior': 1,
         'cpinal': 1,
         'rigerous': 1,
         'multisegmental': 6,
         'reassigned': 3,
         'sensitivities': 5237,
         'rapid-acting': 1,
         'mitazapine': 2,
         'middle/posterior': 1,
         'cp/sob/pnd/claudication': 1,
         'cxr-lll': 2,
         'w/tumor':

### Restrict vocabulary to tokens that appear at least 5 times. Then convert tokens to index numbers -- e.g. "I like pie" becomes [34, 120, 17]  


### ISSUES
- Out-of-Vocabulary words are COMPLETELY IGNORED. In prior assignments, we would often pad unknown words as 0. 
- Furthermore, we might want to link Out-of-Vocabulary words with the nearest token in our dictionary. This would help address misspellings. (Would this be too computationally intensive?)
- Only tokenizing unigrams right now. Will need to tokenize sentences for hierarchical modelling?

In [16]:
vocabulary = create_vocabulary(Corpus_Counts, 5)
HADMID_Text_Dict = {}

for i in range(len(TotalNotes)): #for each discharge summary
    text = TotalNotes['TEXT'][i]
    text_new = preprocess(text)
    tokens = word_tokenize(text_new)
    indexed_document = token_to_index(tokens, vocabulary)
    hadm_id = TotalNotes.loc[i, 'HADM_ID']
    HADMID_Text_Dict[hadm_id] = indexed_document

In [61]:
HADMID_Text_Dict[100006][0:20]

[55966,
 46943,
 25823,
 52900,
 2748,
 17347,
 45907,
 46943,
 25823,
 52900,
 2748,
 17347,
 46943,
 26647,
 54334,
 25823,
 52900,
 2748,
 17347,
 14965]

----------------

## Create One-to-One mapping between HADM_ID and SUBJECT_ID. 
Important because the training/test split will be determined by SubjectID 

In [60]:
IDsOnly = TotalNotes[['HADM_ID','SUBJECT_ID']].drop_duplicates(subset='HADM_ID')
IDsOnly.sort_values(by='SUBJECT_ID').head()

,HADM_ID,SUBJECT_ID
46872,145834.0,3
9714,185777.0,4
25610,107064.0,6
2877,150750.0,9
20917,184167.0,10


--------

## Create DataFrame so that each row is a HADM_ID; columns are SubjectID, Summary Text, and ICD-9 Codes 

In [46]:
HADMID_Text_DF = pd.DataFrame(pd.Series(HADMID_Text_Dict)).rename(columns={0:'Summary'})
HADMID_Code_DF = pd.DataFrame(pd.Series(HADMID_Code_Dict)).rename(columns={0: 'ICD9_Codes'})

In [53]:
print(len(HADMID_Code_DF), len(HADMID_Text_DF))

58929 52726


In [54]:
X = pd.merge(HADMID_Text_DF, HADMID_Code_DF, right_index=True, left_index=True).reset_index(). \
    rename(columns={'index':'HADM_ID'})        
    
TotalChart = pd.merge(IDsOnly, X, on='HADM_ID')

In [59]:
TotalChart.head()

,HADM_ID,SUBJECT_ID,Summary,ICD9_Codes
0,100006,9895,"[55966, 46943, 25823, 52900, 2748, 17347, 4590...","[49320, 51881, 486, 20300, 2761, 7850, 3090, V..."
1,100031,6892,"[55966, 46943, 25823, 52900, 3098, 17347, 4590...","[4412, 4241, 42731, 4538, 5781, 4019, 4430, 73..."
2,100036,30078,"[55966, 46943, 25823, 52900, 9564, 17347, 4590...","[4241, 42833, 9971, 41402, 2762, 4280, 42731, ..."
3,100041,9588,"[55966, 46943, 25823, 52900, 53125, 17347, 459...","[41401, 4414, 412, 496, 2449, 4019, 2749, V1582]"
4,100112,25418,"[55966, 46943, 25823, 52900, 9564, 17347, 4590...","[41401, 4111, 25001, 4019, 2720, V1582]"


----------

# Save chart as JSON file 

In [56]:
TotalChart.to_json('TotalChart.json')

In [58]:
pd.read_json('TotalChart.json').head()

,HADM_ID,ICD9_Codes,SUBJECT_ID,Summary
0,100006,"[49320, 51881, 486, 20300, 2761, 7850, 3090, V...",9895,"[55966, 46943, 25823, 52900, 2748, 17347, 4590..."
1,100031,"[4412, 4241, 42731, 4538, 5781, 4019, 4430, 73...",6892,"[55966, 46943, 25823, 52900, 3098, 17347, 4590..."
10,100154,"[1541, 9974, 5601, 496, 51881, 4280, 48281, 99...",22389,"[55966, 46943, 25823, 52900, 53125, 17347, 459..."
100,101569,"[99662, 03819, 00845, 40391, 51881, 1120, 5849...",17758,"[55966, 46943, 25823, 52900, 53125, 17347, 459..."
1000,116811,"[4240, 5849, 42840, 28731, 4160, 4254, 5185, 9...",27712,"[55966, 46943, 25823, 52900, 3098, 17347, 4590..."


------------

# Optional Section 

In [19]:
### Save HADMID-Text dictionary as a numpy object
np.save('HADM_Text_Dict.npy', HADMID_Text_Dict)

In [20]:
### Save HADMID-ICD9 dictionary as a numpy object
np.save('HADM_Code_Dict.npy', HADMID_Code_Dict)

In [23]:
### Guide to loading numpy objects in Jupyter notebook
sample_dict = {'NYU': 'NY', 'Harvard': 'MA', 'Columbia': 'NY', 'Yale': 'CT'}
np.save('sample_dict.npy', sample_dict)

read_dictionary = np.load('./sample_dict.npy').item()
read_dictionary

{'Columbia': 'NY', 'Harvard': 'MA', 'NYU': 'NY', 'Yale': 'CT'}